In [2]:
import os
import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clip
import glob
from datetime import datetime

def stack_images_sigma_clipped(filter_name, base_dir, output_dir, sigma=3.0, maxiters=5):
    """
    Stack aligned FITS images using sigma-clipped mean.
    
    Parameters:
    -----------
    filter_name : str
        Name of the filter (clear, g, r, i)
    base_dir : str
        Base directory containing filter folders
    output_dir : str
        Directory to save stacked images
    sigma : float
        Number of standard deviations for clipping (default: 3.0)
    maxiters : int
        Maximum number of clipping iterations (default: 5)
    """
    
    print(f"\n{'='*80}")
    print(f"STACKING FILTER: {filter_name.upper()}")
    print(f"{'='*80}")
    
    # Get list of FITS files
    filter_path = os.path.join(base_dir, filter_name)
    fits_files = sorted(glob.glob(os.path.join(filter_path, "*.fits")))
    
    if not fits_files:
        fits_files = sorted(glob.glob(os.path.join(filter_path, "*.fit")))
    
    if len(fits_files) == 0:
        print(f"ERROR: No FITS files found in {filter_path}")
        return None
    
    print(f"Found {len(fits_files)} images to stack")
    print(f"Sigma clipping parameters: sigma={sigma}, maxiters={maxiters}")
    
    # Load all images into a 3D array
    print("\nLoading images into memory...")
    first_image = fits.getdata(fits_files[0])
    img_shape = first_image.shape
    
    # Create 3D array: (num_images, height, width)
    image_stack = np.zeros((len(fits_files), img_shape[0], img_shape[1]), dtype=np.float64)
    
    # Load all images
    for i, fits_file in enumerate(fits_files):
        image_stack[i] = fits.getdata(fits_file)
        if (i + 1) % 10 == 0:
            print(f"  Loaded {i + 1}/{len(fits_files)} images...")
    
    print(f"All {len(fits_files)} images loaded successfully!")
    
    # Perform sigma-clipped stacking
    print(f"\nPerforming sigma-clipped mean stacking...")
    print(f"  This may take a few minutes for {img_shape[0]}x{img_shape[1]} images...")
    
    # Sigma clip along the first axis (across images for each pixel)
    clipped_data = sigma_clip(image_stack, sigma=sigma, maxiters=maxiters, axis=0, masked=True)
    
    # Calculate mean of non-clipped values
    stacked_image = np.ma.mean(clipped_data, axis=0).filled(np.nan)
    
    # Calculate statistics on clipping
    mask_sum = np.sum(clipped_data.mask, axis=0)
    total_pixels = img_shape[0] * img_shape[1]
    clipped_pixels = np.sum(mask_sum)
    clipped_percentage = (clipped_pixels / (len(fits_files) * total_pixels)) * 100
    
    print(f"\nStacking complete!")
    print(f"  Total pixel values: {len(fits_files) * total_pixels:,}")
    print(f"  Clipped values: {clipped_pixels:,} ({clipped_percentage:.2f}%)")
    print(f"  Average values clipped per pixel: {np.mean(mask_sum):.2f}")
    print(f"  Max values clipped for a pixel: {np.max(mask_sum)}")
    
    # Statistics of final stacked image
    print(f"\nStacked image statistics:")
    print(f"  Min value: {np.nanmin(stacked_image):.2f}")
    print(f"  Max value: {np.nanmax(stacked_image):.2f}")
    print(f"  Mean value: {np.nanmean(stacked_image):.2f}")
    print(f"  Median value: {np.nanmedian(stacked_image):.2f}")
    print(f"  Std deviation: {np.nanstd(stacked_image):.2f}")
    
    # Save the stacked image
    os.makedirs(output_dir, exist_ok=True)
    output_filename = os.path.join(output_dir, f"NGC1365_{filter_name}_stacked_sigma{sigma}.fits")
    
    # Copy header from first image and update
    with fits.open(fits_files[0]) as hdul:
        header = hdul[0].header.copy()
    
    # Update header with stacking information
    header['HISTORY'] = f'Sigma-clipped mean stack of {len(fits_files)} images'
    header['HISTORY'] = f'Stacking date: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    header['HISTORY'] = f'Sigma clipping: sigma={sigma}, maxiters={maxiters}'
    header['NIMAGES'] = (len(fits_files), 'Number of images stacked')
    header['STACKMET'] = ('Sigma-clipped mean', 'Stacking method')
    header['SIGMACLP'] = (sigma, 'Sigma clipping threshold')
    header['NCLIPPED'] = (int(clipped_pixels), 'Number of clipped pixel values')
    header['TOTEXP'] = (len(fits_files) * 60.0, 'Total exposure time (seconds)')
    
    # Save as FITS
    hdu = fits.PrimaryHDU(data=stacked_image.astype(np.float64), header=header)
    hdu.writeto(output_filename, overwrite=True)
    
    print(f"\nStacked image saved to: {output_filename}")
    
    return stacked_image, output_filename


def main():
    """Main function to stack all filters."""
    
    # Configuration
    base_dir = "/home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/LIGHT_aligned_cross_correlation"
    output_dir = "/home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED"
    
    filters = ['clear', 'g', 'r', 'i']
    
    # Sigma clipping parameters
    sigma = 3.0  # Standard sigma clipping threshold
    maxiters = 5  # Maximum iterations for clipping
    
    print("="*80)
    print("SIGMA-CLIPPED IMAGE STACKING")
    print("="*80)
    print(f"Target: NGC1365")
    print(f"Base directory: {base_dir}")
    print(f"Output directory: {output_dir}")
    print(f"Filters to process: {', '.join(filters)}")
    print(f"Stacking method: Sigma-clipped mean")
    print(f"Sigma threshold: {sigma}")
    print(f"Max iterations: {maxiters}")
    
    # Stack each filter
    results = {}
    start_time = datetime.now()
    
    for filter_name in filters:
        try:
            stacked_img, output_file = stack_images_sigma_clipped(
                filter_name, base_dir, output_dir, sigma=sigma, maxiters=maxiters
            )
            results[filter_name] = {
                'success': True,
                'output_file': output_file,
                'stats': {
                    'min': np.nanmin(stacked_img),
                    'max': np.nanmax(stacked_img),
                    'mean': np.nanmean(stacked_img),
                    'median': np.nanmedian(stacked_img)
                }
            }
        except Exception as e:
            print(f"\nERROR processing {filter_name}: {str(e)}")
            results[filter_name] = {'success': False, 'error': str(e)}
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    
    # Summary
    print(f"\n{'='*80}")
    print("STACKING SUMMARY")
    print(f"{'='*80}")
    print(f"Total processing time: {duration:.1f} seconds ({duration/60:.1f} minutes)")
    print(f"\nSuccessfully stacked filters:")
    
    for filter_name, result in results.items():
        if result['success']:
            print(f"  ✓ {filter_name.upper()}: {result['output_file']}")
    
    failed = [f for f, r in results.items() if not r['success']]
    if failed:
        print(f"\nFailed filters: {', '.join(failed)}")
    
    print(f"\n{'='*80}")
    print("STACKING COMPLETE!")
    print(f"{'='*80}")


if __name__ == "__main__":
    main()

SIGMA-CLIPPED IMAGE STACKING
Target: NGC1365
Base directory: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/LIGHT_aligned_cross_correlation
Output directory: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED
Filters to process: clear, g, r, i
Stacking method: Sigma-clipped mean
Sigma threshold: 3.0
Max iterations: 5

STACKING FILTER: CLEAR
Found 62 images to stack
Sigma clipping parameters: sigma=3.0, maxiters=5

Loading images into memory...
  Loaded 10/62 images...
  Loaded 20/62 images...
  Loaded 30/62 images...
  Loaded 40/62 images...
  Loaded 50/62 images...
  Loaded 60/62 images...
All 62 images loaded successfully!

Performing sigma-clipped mean stacking...
  This may take a few minutes for 2048x2048 images...

Stacking complete!
  Total pixel values: 260,046,848
  Clipped values: 703,960 (0.27%)
  Average values clipped per pixel: 0.17
  Max values clipped for a pixel: 16


In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clip
from astropy.visualization import ZScaleInterval, ImageNormalize, LinearStretch
import matplotlib.pyplot as plt
import glob
from datetime import datetime

def stack_images_sigma_clipped(filter_name, base_dir, output_dir, sigma=3.0, maxiters=5):
    """
    Stack aligned FITS images using sigma-clipped mean.
    
    Parameters:
    -----------
    filter_name : str
        Name of the filter (clear, g, r, i)
    base_dir : str
        Base directory containing filter folders
    output_dir : str
        Directory to save stacked images
    sigma : float
        Number of standard deviations for clipping (default: 3.0)
    maxiters : int
        Maximum number of clipping iterations (default: 5)
    """
    
    print(f"\n{'='*80}")
    print(f"STACKING FILTER: {filter_name.upper()}")
    print(f"{'='*80}")
    
    # Get list of FITS files
    filter_path = os.path.join(base_dir, filter_name)
    fits_files = sorted(glob.glob(os.path.join(filter_path, "*.fits")))
    
    if not fits_files:
        fits_files = sorted(glob.glob(os.path.join(filter_path, "*.fit")))
    
    if len(fits_files) == 0:
        print(f"ERROR: No FITS files found in {filter_path}")
        return None, None, None
    
    print(f"Found {len(fits_files)} images to stack")
    print(f"Sigma clipping parameters: sigma={sigma}, maxiters={maxiters}")
    
    # Load all images into a 3D array
    print("\nLoading images into memory...")
    first_image = fits.getdata(fits_files[0])
    img_shape = first_image.shape
    
    # Create 3D array: (num_images, height, width)
    image_stack = np.zeros((len(fits_files), img_shape[0], img_shape[1]), dtype=np.float64)
    
    # Load all images
    for i, fits_file in enumerate(fits_files):
        image_stack[i] = fits.getdata(fits_file)
        if (i + 1) % 10 == 0:
            print(f"  Loaded {i + 1}/{len(fits_files)} images...")
    
    print(f"All {len(fits_files)} images loaded successfully!")
    
    # Perform sigma-clipped stacking
    print(f"\nPerforming sigma-clipped mean stacking...")
    print(f"  This may take a few minutes for {img_shape[0]}x{img_shape[1]} images...")
    
    # Sigma clip along the first axis (across images for each pixel)
    clipped_data = sigma_clip(image_stack, sigma=sigma, maxiters=maxiters, axis=0, masked=True)
    
    # Calculate mean of non-clipped values
    stacked_image = np.ma.mean(clipped_data, axis=0).filled(np.nan)
    
    # Calculate statistics on clipping
    mask_sum = np.sum(clipped_data.mask, axis=0)
    total_pixels = img_shape[0] * img_shape[1]
    clipped_pixels = np.sum(mask_sum)
    clipped_percentage = (clipped_pixels / (len(fits_files) * total_pixels)) * 100
    
    print(f"\nStacking complete!")
    print(f"  Total pixel values: {len(fits_files) * total_pixels:,}")
    print(f"  Clipped values: {clipped_pixels:,} ({clipped_percentage:.2f}%)")
    print(f"  Average values clipped per pixel: {np.mean(mask_sum):.2f}")
    print(f"  Max values clipped for a pixel: {np.max(mask_sum)}")
    
    # Statistics of final stacked image
    print(f"\nStacked image statistics:")
    print(f"  Min value: {np.nanmin(stacked_image):.2f}")
    print(f"  Max value: {np.nanmax(stacked_image):.2f}")
    print(f"  Mean value: {np.nanmean(stacked_image):.2f}")
    print(f"  Median value: {np.nanmedian(stacked_image):.2f}")
    print(f"  Std deviation: {np.nanstd(stacked_image):.2f}")
    
    # Save the stacked image
    os.makedirs(output_dir, exist_ok=True)
    output_filename = os.path.join(output_dir, f"NGC1365_{filter_name}_stacked_sigma{sigma}.fits")
    
    # Copy header from first image and update
    with fits.open(fits_files[0]) as hdul:
        header = hdul[0].header.copy()
    
    # Update header with stacking information
    header['HISTORY'] = f'Sigma-clipped mean stack of {len(fits_files)} images'
    header['HISTORY'] = f'Stacking date: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
    header['HISTORY'] = f'Sigma clipping: sigma={sigma}, maxiters={maxiters}'
    header['NIMAGES'] = (len(fits_files), 'Number of images stacked')
    header['STACKMET'] = ('Sigma-clipped mean', 'Stacking method')
    header['SIGMACLP'] = (sigma, 'Sigma clipping threshold')
    header['NCLIPPED'] = (int(clipped_pixels), 'Number of clipped pixel values')
    header['TOTEXP'] = (len(fits_files) * 60.0, 'Total exposure time (seconds)')
    
    # Save as FITS
    hdu = fits.PrimaryHDU(data=stacked_image.astype(np.float64), header=header)
    hdu.writeto(output_filename, overwrite=True)
    
    print(f"\nStacked image saved to: {output_filename}")
    
    return stacked_image, output_filename, first_image


def visualize_comparison(filter_name, raw_image, stacked_image, output_dir):
    """
    Create visualization comparing raw (first frame) vs stacked image.
    
    Parameters:
    -----------
    filter_name : str
        Name of the filter
    raw_image : ndarray
        First raw frame for comparison
    stacked_image : ndarray
        Final stacked image
    output_dir : str
        Directory to save visualization
    """
    
    print(f"\nCreating visualization for {filter_name}...")
    
    # Use ZScale for better astronomical image display
    zscale = ZScaleInterval()
    
    # Get scaling limits from stacked image (usually has better signal)
    vmin_stack, vmax_stack = zscale.get_limits(stacked_image)
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(f'NGC1365 - {filter_name.upper()} Filter: Raw vs Stacked Comparison', 
                 fontsize=16, fontweight='bold')
    
    # Row 1: Full images
    # Raw image (first frame)
    im1 = axes[0, 0].imshow(raw_image, cmap='gray', origin='lower', 
                            vmin=vmin_stack, vmax=vmax_stack)
    axes[0, 0].set_title('Single Raw Frame (First Image)', fontsize=12)
    axes[0, 0].set_xlabel('X (pixels)')
    axes[0, 0].set_ylabel('Y (pixels)')
    plt.colorbar(im1, ax=axes[0, 0], fraction=0.046, pad=0.04, label='ADU')
    
    # Stacked image
    im2 = axes[0, 1].imshow(stacked_image, cmap='gray', origin='lower',
                            vmin=vmin_stack, vmax=vmax_stack)
    axes[0, 1].set_title('Stacked Image (Sigma-Clipped Mean)', fontsize=12)
    axes[0, 1].set_xlabel('X (pixels)')
    axes[0, 1].set_ylabel('Y (pixels)')
    plt.colorbar(im2, ax=axes[0, 1], fraction=0.046, pad=0.04, label='ADU')
    
    # Difference image (showing improvement)
    diff_image = stacked_image - raw_image
    im3 = axes[0, 2].imshow(diff_image, cmap='RdBu_r', origin='lower')
    axes[0, 2].set_title('Difference (Stacked - Raw)', fontsize=12)
    axes[0, 2].set_xlabel('X (pixels)')
    axes[0, 2].set_ylabel('Y (pixels)')
    plt.colorbar(im3, ax=axes[0, 2], fraction=0.046, pad=0.04, label='ADU')
    
    # Row 2: Zoomed regions (center of galaxy)
    center_y, center_x = stacked_image.shape[0] // 2, stacked_image.shape[1] // 2
    zoom_size = 300  # 300x300 pixel zoom
    
    y1, y2 = center_y - zoom_size//2, center_y + zoom_size//2
    x1, x2 = center_x - zoom_size//2, center_x + zoom_size//2
    
    # Zoomed raw
    raw_zoom = raw_image[y1:y2, x1:x2]
    im4 = axes[1, 0].imshow(raw_zoom, cmap='gray', origin='lower',
                            vmin=vmin_stack, vmax=vmax_stack)
    axes[1, 0].set_title(f'Raw Frame (Center {zoom_size}x{zoom_size}px)', fontsize=12)
    axes[1, 0].set_xlabel('X (pixels)')
    axes[1, 0].set_ylabel('Y (pixels)')
    plt.colorbar(im4, ax=axes[1, 0], fraction=0.046, pad=0.04, label='ADU')
    
    # Zoomed stacked
    stacked_zoom = stacked_image[y1:y2, x1:x2]
    im5 = axes[1, 1].imshow(stacked_zoom, cmap='gray', origin='lower',
                            vmin=vmin_stack, vmax=vmax_stack)
    axes[1, 1].set_title(f'Stacked Image (Center {zoom_size}x{zoom_size}px)', fontsize=12)
    axes[1, 1].set_xlabel('X (pixels)')
    axes[1, 1].set_ylabel('Y (pixels)')
    plt.colorbar(im5, ax=axes[1, 1], fraction=0.046, pad=0.04, label='ADU')
    
    # SNR comparison plot
    axes[1, 2].axis('off')
    
    # Calculate approximate SNR improvement
    raw_std = np.std(raw_image[100:200, 100:200])  # Background region
    stacked_std = np.std(stacked_image[100:200, 100:200])
    snr_improvement = raw_std / stacked_std
    
    # Statistics text
    stats_text = f"""STACKING RESULTS
    
Raw Frame Statistics:
  Mean: {np.nanmean(raw_image):.2f}
  Median: {np.nanmedian(raw_image):.2f}
  Std Dev: {np.nanstd(raw_image):.2f}
  
Stacked Image Statistics:
  Mean: {np.nanmean(stacked_image):.2f}
  Median: {np.nanmedian(stacked_image):.2f}
  Std Dev: {np.nanstd(stacked_image):.2f}
  
Improvement:
  Background Noise Reduction: {snr_improvement:.2f}x
  Expected from √N: {np.sqrt(62):.2f}x
  
The stacked image shows:
  • Reduced noise in background
  • Enhanced faint structures
  • Better signal-to-noise ratio
  • Removed cosmic rays/outliers"""
    
    axes[1, 2].text(0.1, 0.5, stats_text, fontsize=10, family='monospace',
                    verticalalignment='center', transform=axes[1, 2].transAxes,
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    
    # Save visualization
    vis_filename = os.path.join(output_dir, f"NGC1365_{filter_name}_comparison.png")
    plt.savefig(vis_filename, dpi=150, bbox_inches='tight')
    print(f"Visualization saved to: {vis_filename}")
    
    plt.close()


def main():
    """Main function to stack all filters."""
    
    # Configuration
    base_dir = "/home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/LIGHT_aligned_cross_correlation"
    output_dir = "/home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED"
    
    filters = ['clear', 'g', 'r', 'i']
    
    # Sigma clipping parameters
    sigma = 3.0  # Standard sigma clipping threshold
    maxiters = 5  # Maximum iterations for clipping
    
    print("="*80)
    print("SIGMA-CLIPPED IMAGE STACKING WITH VISUALIZATION")
    print("="*80)
    print(f"Target: NGC1365")
    print(f"Base directory: {base_dir}")
    print(f"Output directory: {output_dir}")
    print(f"Filters to process: {', '.join(filters)}")
    print(f"Stacking method: Sigma-clipped mean")
    print(f"Sigma threshold: {sigma}")
    print(f"Max iterations: {maxiters}")
    
    # Stack each filter
    results = {}
    start_time = datetime.now()
    
    for filter_name in filters:
        try:
            stacked_img, output_file, raw_img = stack_images_sigma_clipped(
                filter_name, base_dir, output_dir, sigma=sigma, maxiters=maxiters
            )
            
            # Create visualization
            visualize_comparison(filter_name, raw_img, stacked_img, output_dir)
            
            results[filter_name] = {
                'success': True,
                'output_file': output_file,
                'stats': {
                    'min': np.nanmin(stacked_img),
                    'max': np.nanmax(stacked_img),
                    'mean': np.nanmean(stacked_img),
                    'median': np.nanmedian(stacked_img)
                }
            }
        except Exception as e:
            print(f"\nERROR processing {filter_name}: {str(e)}")
            import traceback
            traceback.print_exc()
            results[filter_name] = {'success': False, 'error': str(e)}
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    
    # Summary
    print(f"\n{'='*80}")
    print("STACKING SUMMARY")
    print(f"{'='*80}")
    print(f"Total processing time: {duration:.1f} seconds ({duration/60:.1f} minutes)")
    print(f"\nSuccessfully stacked filters:")
    
    for filter_name, result in results.items():
        if result['success']:
            print(f"  ✓ {filter_name.upper()}: {result['output_file']}")
            print(f"      Visualization: {output_dir}/NGC1365_{filter_name}_comparison.png")
    
    failed = [f for f, r in results.items() if not r['success']]
    if failed:
        print(f"\nFailed filters: {', '.join(failed)}")
    
    print(f"\n{'='*80}")
    print("STACKING COMPLETE!")
    print(f"{'='*80}")
    print(f"\nCheck the '{output_dir}' folder for:")
    print(f"  - Stacked FITS files: NGC1365_<filter>_stacked_sigma3.fits")
    print(f"  - Comparison images: NGC1365_<filter>_comparison.png")


if __name__ == "__main__":
    main()

SIGMA-CLIPPED IMAGE STACKING WITH VISUALIZATION
Target: NGC1365
Base directory: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/LIGHT_aligned_cross_correlation
Output directory: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED
Filters to process: clear, g, r, i
Stacking method: Sigma-clipped mean
Sigma threshold: 3.0
Max iterations: 5

STACKING FILTER: CLEAR
Found 62 images to stack
Sigma clipping parameters: sigma=3.0, maxiters=5

Loading images into memory...
  Loaded 10/62 images...
  Loaded 20/62 images...
  Loaded 30/62 images...
  Loaded 40/62 images...
  Loaded 50/62 images...
  Loaded 60/62 images...
All 62 images loaded successfully!

Performing sigma-clipped mean stacking...
  This may take a few minutes for 2048x2048 images...

Stacking complete!
  Total pixel values: 260,046,848
  Clipped values: 703,960 (0.27%)
  Average values clipped per pixel: 0.17
  Max values clipp